## One possible solution:
This example highlights the importance of understanding the limitations of transformer models when dealing with long texts. Simply throwing a long text at a transformer model without considering its token length limit will likely result in failure.

In the next section of this notebook, we'll explore strategies for effectively handling long texts in transformer models for QA tasks.

## Handling Long Texts with Transformers: A Practical Approach

In the previous section, we saw how naively applying a transformer model to a long text can lead to errors. But don't worry! We've developed a set of Python functions as possible solution for handling long texts in transformer models for QA tasks.

While the approach we've outlined below is a practical solution for handling long texts with transformer models, it's by no means the only or the best solution. There are several ways you could improve upon our code or implement your own solution.

For instance, you could experiment with different chunk sizes and overlaps to see if they yield better results. You could also try different ways of ranking the answers, such as using different similarity measures or incorporating additional information into the ranking process.

In addition, there are many existing repositories on the internet that provide solutions for handling long texts with transformer models. These repositories often include more advanced techniques and optimizations, and can be a great resource for improving your own implementation.

Remember, the key to success in machine learning is experimentation and continuous learning. Don't be afraid to try new things and learn from others. 

### Splitting Text into Chunks
To overcome the token length limit of transformer models, we split the long text into smaller chunks (see "split_context" below). Each chunk is of a specific size that the model can handle, and the chunks overlap a bit to ensure we don't miss any information that could be at the boundaries of the chunks.

### Asking Questions on Each Chunk
We then feed each chunk to the model and collect the answers (see "answer_question" below). This ensures that the model considers the question in the context of each chunk of the long text.

### Ranking the Answers
Once we have the answers, we need a way to determine which one is the best (see "find_best_answer" below). For this, we use the concept of vectorizing the answers and the question (see "vectorize_text" below), and then calculating the cosine similarity between them (see "calculate_similarity" below). The answer with the highest cosine similarity to the question is considered the best answer. 

This approach allows us to leverage the power of transformer models for QA tasks on long texts, without running into memory issues or the need for complex fine-tuning. It's a practical and effective solution for tasks like log file analysis, where the texts can be very long.



In [70]:
import numpy as np
from scipy.spatial.distance import cosine
import torch

def is_text_redundant(text):
    """
    Check if a text is redundant.

    A simple approach to check redundancy is to look for repeated phrases or sentences.
    This function uses a basic method where it checks for repeated sequences of words.
    
    param text: The text to be checked for redundancy.
    return: True if the text is redundant, False otherwise.
    """
    words = text.split()
    seen = set()
    for i in range(len(words) - 5):  # Check sequences of 5 words
        sequence = ' '.join(words[i:i+5])
        if sequence in seen:
            return True
        seen.add(sequence)
    return False

def determine_overlap(text, max_length):
    """
    Determine the best overlapping parameter for splitting a text.

    The best overlap is determined based on the length and redundancy of the text.
    Shorter and less redundant texts can have smaller overlaps, while longer and more redundant
    texts may require larger overlaps to ensure continuity.

    param text: The text for which to determine the best overlap.
    param max_length: The maximum length of each chunk in characters.
    return: The recommended overlap size in characters.
    """
    length = len(text)

    # Set base overlap sizes
    short_text_overlap = 20  # For short, non-redundant texts
    long_text_overlap = 50  # For long or redundant texts

    if length < max_length:
        return short_text_overlap
    else:
        if is_text_redundant(text):
            # If the text is redundant, increase the overlap to handle complexity
            return long_text_overlap + 30  # Increasing overlap for redundancy
        else:
            # For longer texts which are not redundant
            return long_text_overlap

def split_context(context, max_length=512):
    """
    Function for splitting context into overlapping chunks.
    
    param context: This is the text that you want to split into chunks. 
    The function will split this text based on the max_length and overlap parameters.

    param overlap (default=50): This is the number of characters that will overlap between each chunk. 
    This is used to ensure that the context is not cut off in the middle of a sentence, which could make the text difficult to understand.

    param max_length (default=512): This is the maximum length of each chunk. 
    The function will split the context into chunks of this length, with the exception of the last chunk, which may be shorter.

    The function returns a list of chunks, where each chunk is a string of text from the context. 
    The chunks are created by starting at the beginning of the context and moving forward max_length
    characters at a time, with an overlap of overlap characters between each chunk.
    """
    overlap = determine_overlap(context, max_length)
    
    chunks = []
    start = 0
    while start < len(context):
        end = min(start + max_length, len(context))
        chunks.append(context[start:end])
        if end == len(context):
            break
        start = end - overlap
    return chunks


def answer_question(model, tokenizer, context, max_length, question):
    """
    The function answers questions given context and question.
    
    model: This is the model that you're using to generate answers to the questions. 
    It could be any model that's capable of question answering, such as a transformer model.

    param  tokenizer: This is the tokenizer that corresponds to your model. 
    It's used to convert your text data into a format that the model can understand.

    param context: This is the text that the model will look at to find an answer to the question.

    param question: This is the question that you're asking the model. 
    The model will generate an answer to this question based on the context.

    The function returns an answer to the question based on the context. 
    The answer is generated by finding the tokens with the highest start and end scores, 
    and joining them together. If the end score is higher than the start score,  
    they are swapped to ensure the answer makes sense.
    """
    # Encode the context and question
    encoded = tokenizer.encode_plus(question, context, truncation=True, padding='max_length', max_length= max_length, return_tensors='pt')

    # Get the start and end scores for all tokens
    result = model(**encoded)
    start_scores = result["start_logits"]
    end_scores = result["end_logits"]

    # Find the tokens with the highest start and end scores
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # If the end score is higher than the start score, swap them
    if answer_end < answer_start:
        answer_start, answer_end = answer_end, answer_start

    # Get the tokens for the answer
    all_tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])
    answer = ' '.join(all_tokens[answer_start : answer_end+1])

    return answer


def vectorize_text(model, tokenizer, input_string):
    """
    Vectorize a given input string.
    
    param model: This is the model used to encode the input string and get the output. 
    It could be any model that's capable of encoding text, such as a transformer model.

    param tokenizer: This is the tokenizer that corresponds to your model. 
    It's used to convert your text data into a format that the model can understand.

    param input_string: This is the text that you want to vectorize. 
    The function will convert this text into a numerical representation that 
    can be processed by the machine learning model.

    The function returns a vector representation of the input string. 
    This vector is obtained by averaging the embeddings from the last hidden 
    state of the model's output.
    """
    # Encode the input string
    inputs = tokenizer.encode_plus(
        input_string,
        add_special_tokens=True,
        return_tensors="pt"
    )

    # Get the output from the model
    outputs = model(**inputs)

    # Get the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state

    # Average the embeddings
    vector = torch.mean(embeddings, dim=1)

    # Convert tensor to numpy array
    vector = vector.detach().numpy()

    return vector


def calculate_similarity(question_vector, answer_vector):
    """Calculate the cosine similarity between the question and answer vectors.
    
    param question_vector: This is the vector representation of the question. 
    It's obtained by transforming the question text into numerical data that 
    can be processed by the machine learning model.

    param answer_vector: This is the vector representation of the answer. 
    It's obtained by transforming the answer text into numerical data that 
    can be processed by the machine learning model.

    The function calculates and returns the cosine similarity between the 
    question and answer vectors. Cosine similarity is a measure of similarity 
    between two non-zero vectors of an inner product space that measures the 
    cosine of the angle between them. The closer the cosine similarity to 1, 
    the more similar the question and answer are.
    """

    similarity = 1 - cosine(question_vector[0], answer_vector[0])

    return similarity


def find_best_answer(model, tokenizer, context, question, model_vec, num_answers=3, max_length=512):
    """Find the best answers to the question given a long context
    param model: This is the model that you're using to generate answers to the questions. 
    It could be any model that's capable of question answering, such as a transformer model.

    param tokenizer: This is the tokenizer that corresponds to your model.
    It's used to convert your text data into a format that the model can understand.

    param context: This is the text that the model will look at to find an answer to the question. 
    In this case, it's a long text that's split into chunks.

    param question: This is the question that you're asking the model. 
    The model will generate an answer to this question based on the context.

    param model_vec: This is a model used to vectorize the text, 
    i.e., convert the text into numerical data that can be processed by the machine learning model.

    param num_answers (default=3): This is the number of best answers the function will return.

    param overlap (default=50): This is the number of overlapping words between 
    two consecutive chunks when the context is split into chunks.

    param max_length (default=512): This is the maximum length of each chunk. 
    The context is split into chunks of this length.

    The function returns a list of tuples, where each tuple contains an answer 
    and its similarity score. The list is sorted in ascending order of similarity, 
    so the first element of the list is the answer with the lowest similarity, 
    and the last element is the answer with the highest similarity.
        
    """
    # Vectorize the question
    question_vector = vectorize_text(model_vec, tokenizer, question)
    
    # Initialize the best answers and their similarities to the question
    best_answers = [(None, -1) for _ in range(num_answers)]
    
    # Split the context into chunks
    chunks = split_context(context,max_length)
    
    # Generate an answer for each chunk and update the best answers if necessary
    for chunk in chunks:
        answer = answer_question(model, tokenizer, chunk, max_length, question)
        if answer is not None:
            answer_vector = vectorize_text(model_vec, tokenizer, answer)
    
            if answer_vector is not None:
                similarity = calculate_similarity(question_vector, answer_vector)
                # Check if the similarity is higher than the current lowest in best_answers
                if similarity > best_answers[0][1]:
                    # Replace the lowest
                    best_answers[0] = (answer, similarity)
                    # Sort the list so the lowest similarity is first
                    best_answers = sorted(best_answers, key=lambda x: x[1])
    # Return the answers along with their similarities
    return best_answers


In [71]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel

model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = AutoModelForQuestionAnswering.from_pretrained(model_name) #used for answering
model_vec = AutoModel.from_pretrained(model_name) #used for vectorization
#model_kmeans = AutoModel.from_pretrained(model_name) #used for clustering
#model_summarizer = AutoModel.from_pretrained(model_name) #used for summarization
tokenizer = AutoTokenizer.from_pretrained(model_name) #used for tokenization


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Example context and question
context = """germany,[e] officially the Federal Republic of Germany,[f] is a country in the western region of Central Europe. It is the second-most populous country in Europe after Russia, and the most populous member state of the European Union. Germany is situated between the Baltic and North seas to the north, and the Alps to the south. Its 16 constituent states are bordered by Denmark to the north, Poland and the Czech Republic to the east, Austria and Switzerland to the south, and France, Luxembourg, Belgium, and the Netherlands to the west. The nation's capital and most populous city is Berlin and its main financial centre is Frankfurt; the largest urban area is the Ruhr.

Various Germanic tribes have inhabited the northern parts of modern Germany since classical antiquity. A region named Germania was documented before AD 100. In 962, the Kingdom of Germany formed the bulk of the Holy Roman Empire. During the 16th century, northern German regions became the centre of the Protestant Reformation. Following the Napoleonic Wars and the dissolution of the Holy Roman Empire in 1806, the German Confederation was formed in 1815.

Formal unification of Germany into the modern nation-state was commenced on 18 August 1866 with the North German Confederation Treaty establishing the Prussia-led North German Confederation later transformed in 1871 into the German Empire. After World War I and the German Revolution of 1918–1919, the Empire was in turn transformed into the semi-presidential Weimar Republic. The Nazi seizure of power in 1933 led to the establishment of a totalitarian dictatorship, World War II, and the Holocaust. After the end of World War II in Europe and a period of Allied occupation, in 1949, Germany as a whole was organized into two separate polities with limited sovereignty: the Federal Republic of Germany, generally known as West Germany, and the German Democratic Republic, known as East Germany, while Berlin continued its de jure Four Power status. The Federal Republic of Germany was a founding member of the European Economic Community and the European Union, while the German Democratic Republic was a communist Eastern Bloc state and member of the Warsaw Pact. After the fall of communist led-government in East Germany, German reunification saw the former East German states join the Federal Republic of Germany on 3 October 1990.

Germany has been described as a great power with a strong economy; it has the largest economy in Europe, the world's third-largest economy by nominal GDP and the fifth-largest by PPP. As a global power in industrial, scientific and technological sectors, it is both the world's third-largest exporter and importer. As a developed country it offers social security, a universal health care system and a tuition-free university education. Germany is a member of the United Nations, European Union, NATO, Council of Europe, G7, G20, and OECD. It has the third-greatest number of UNESCO World Heritage Sites."""
question = "Is germany part of the EU?"


# Find the best answer to the question given the context
best_answer = find_best_answer(model, tokenizer, 
                               context, question, 
                               model_vec, 
                               num_answers=3, 
                               max_length=tokenizer.model_max_length)
print(f"The best answers are: {best_answer}")

The best answers are: [('the most populous member state of the european union', 0.8635246157646179), ('germany is a member of the united nations , european union', 0.866646945476532), ('te ##s join the federal republic of germany', 0.8742250204086304)]


In [73]:
context = """
INFO - 2022-01-01 00:00:01,234 - Connection established.
INFO - 2022-01-01 00:00:02,345 - Sending request to server.
ERROR - 2022-01-01 00:00:03,456 - Failed to establish a new connection: [Errno 11001] getaddrinfo failed.
INFO - 2022-01-01 00:00:04,567 - Connection closed.
INFO - 2022-01-01 00:00:05,678 - Attempting to reconnect.
INFO - 2022-01-01 00:00:06,789 - Connection established.
INFO - 2022-01-01 00:00:07,890 - Sending request to server.
INFO - 2022-01-01 00:00:08,901 - Server response received.
INFO - 2022-01-01 00:00:09,012 - Processing server response.
ERROR - 2022-01-01 00:00:10,123 - Error while processing server response: Unexpected token.
INFO - 2022-01-01 00:00:11,234 - Connection closed.
"""

#question = "What error message is displayed?"
question = "How many errors are in the file?"

# Find the best answer to the question given the context
best_answer = find_best_answer(model, tokenizer, context, question, model_vec, num_answers=3, max_length=tokenizer.model_max_length)
print(f"The best answer is: {best_answer}")

The best answer is: [(None, -1), ('202 ##2 - 01 - 01 00 : 00 : 08 , 90 ##1', 0.7278519868850708), ('[CLS] how many errors are in the file ? [SEP] info - 202 ##2 - 01 - 01 00 : 00 : 01 , 234', 0.813469648361206)]


In [19]:
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
import nltk
nltk.download('punkt')


def vectorize_texts(texts, embedding_model):
    """
    Vectorizes texts using word embeddings.

    :param texts: A list of text documents.
    :param embedding_model: A pre-trained word embedding model from gensim.
    :return: A list of vectorized text representations.
    """
    vectorized_texts = []

    for text in texts:
        # Tokenize and remove stopwords
        words = nltk.word_tokenize(remove_stopwords(text.lower()))
        # Filter words in the model's vocabulary
        words = [word for word in words if word in embedding_model.key_to_index]

        if words:
            # Average the word vectors for the text
            text_vector = np.mean([embedding_model[word] for word in words], axis=0)
        else:
            # Handle case for texts with no words in the model's vocabulary
            text_vector = np.zeros(embedding_model.vector_size)
        
        vectorized_texts.append(text_vector)

    return vectorized_texts


[nltk_data] Downloading package punkt to /home/hackathon1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
import gensim.downloader
# Download the model and return as object ready for use
embedding_model = gensim.downloader.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [35]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from transformers import BartTokenizer, BartForConditionalGeneration

def cluster_and_summarize(model, tokenizer, model_vect, texts, vectorized_texts, question, num_clusters):
    """
    Clusters texts and then summarizes the representative chunk of each cluster using BART.

    :param texts: A list of original text documents.
    :param vectorized_texts: Vectorized representations of the texts.
    :param num_clusters: Number of clusters for k-means.
    :return: A list of summaries, one for each cluster.
    """
    # Vectorize the question
    question_vector = vectorize_texts(question,model_vect)

    # Ensure that num_clusters is at least 2
    num_clusters = max(num_clusters, 2)

    # Initialize K-means with custom initial centers
    initial_centers = np.vstack([question_vector, np.random.rand(num_clusters - 1, question_vector.shape[1])])
    kmeans = KMeans(n_clusters=num_clusters, init=initial_centers, n_init=1, random_state=42)
    kmeans.fit(vectorized_texts)
    
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_

    representative_texts = []
    for i in range(num_clusters):
        # Find texts in the current cluster
        cluster_texts = [vectorized_texts[j] for j in range(len(texts)) if labels[j] == i]

        # Calculate distances of all points in this cluster to the centroid
        distances = cdist(cluster_texts, [centroids[i]], 'euclidean')

        # Find the text closest to the centroid
        min_index = np.argmin(distances)
        representative_text = texts[min_index]
        
        # Combine the question with the representative text
        combined_text = question + " " + representative_text
        representative_texts.append(combined_text)

    summaries = []
    for text in representative_texts:
        inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
        
    return summaries


In [47]:
def answer_question_from_summaries(model, tokenizer, question, summaries):
    """
    Answers a question based on a set of text summaries.

    :param question: A string containing the question.
    :param summaries: A list of text summaries.
    :return: The answer to the question based on the summaries.
    """

    for summary in summaries:
        # Encode question and summary
        inputs = tokenizer.encode_plus(question, summary, add_special_tokens=True, return_tensors="pt")
        input_ids = inputs["input_ids"].tolist()[0]

        # Check if the length of the input is manageable for the model
        if len(input_ids) > 512:
            continue

        # Get model's answer
        answer_start_scores, answer_end_scores = model(**inputs)
        print(type(answer_start_scores))

        #Find the position of the start and end of the answer in the text
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        # Convert tokens to answer string
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
        
        if answer:
            return answer

    return "No answer found."

In [60]:
def answer_question_based_on_summaries(question, summaries):
    """
    Answers a question based on a set of text summaries.

    :param question: A string containing the question.
    :param summaries: A list of text summaries.
    :return: The answer to the question based on the summaries.
    """
    # Load the question-answering model and tokenizer
    model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    for summary in summaries:
        # Encode the question and summary
        inputs = tokenizer.encode_plus(question, summary, add_special_tokens=True, return_tensors="pt")
        input_ids = inputs["input_ids"].tolist()[0]

        # Forward pass, get model output
        outputs = model(**inputs)

        # The model outputs start and end scores for each token in the text
        answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits

        # Find the position of the start and end of the answer
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        # Convert the tokens to the answer string
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
        
        if answer:
            # Print the answer
            print("Answer:", answer)
            return

    print("No answer found within the summaries.")

In [37]:
# Summarizing model
tokenizer_summary = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model_summary = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

#Answering model 
model_answer = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
tokenizer_answer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad") #used for tokenization

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [63]:
context = """[Sun Jan 15 04:11:23 2006] [notice] Digest: generating secret for digest authentication ...
[Sun Jan 15 04:11:23 2006] [notice] Digest: done
[Sun Jan 15 04:11:23 2006] [notice] LDAP: Built with OpenLDAP LDAP SDK
[Sun Jan 15 04:11:23 2006] [notice] LDAP: SSL support unavailable
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating channel.jni:jni ( channel.jni, jni)
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create channel.jni:jni
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating vm: ( vm, )
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create vm:
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating worker.jni:onStartup ( worker.jni, onStartup)
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create worker.jni:onStartup
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating worker.jni:onShutdown ( worker.jni, onShutdown)
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create worker.jni:onShutdown
[Sun Jan 15 04:11:25 2006] [notice] mod_python: Creating 32 session mutexes based on 150 max processes and 0 max threads.
[Sun Jan 15 04:11:25 2006] [notice] mod_security/1.9dev2 configured
[Sun Jan 15 04:11:25 2006] [notice] Apache/2.0.49 (Fedora) configured -- resuming normal operations
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27313 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27314 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27316 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27317 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27318 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27319 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] jk2_init() Found child 27315 in scoreboard slot 13
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [notice] jk2_init() Found child 27320 in scoreboard slot 1
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 04:17:29 2006] [error] [client 222.78.70.94] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 04:25:54 2006] [error] [client 68.193.179.224] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 05:29:33 2006] [error] [client 203.196.138.5] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 06:23:20 2006] [error] [client 66.214.157.114] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 06:53:16 2006] [error] [client 218.204.92.66] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 07:56:06 2006] [error] [client 64.60.3.98] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 08:48:10 2006] [error] [client 69.220.16.126] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 10:51:33 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:33 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] jk2_init() Can't find child 32229 in scoreboard
[Sun Jan 15 10:51:34 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:34 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:35 2006] [notice] jk2_init() Found child 32230 in scoreboard slot 2
[Sun Jan 15 10:51:35 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:35 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:36 2006] [error] jk2_init() Can't find child 32231 in scoreboard
[Sun Jan 15 10:51:36 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:36 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:37 2006] [error] jk2_init() Can't find child 32232 in scoreboard
[Sun Jan 15 10:51:37 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:37 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
[Sun Jan 15 10:51:38 2006] [notice] jk2_init() Found child 32233 in scoreboard slot 6
[Sun Jan 15 10:51:38 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:38 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:39 2006] [error] jk2_init() Can't find child 32234 in scoreboard
[Sun Jan 15 10:51:39 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:39 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
script not found or unable to stat
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
script not found or unable to stat
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
[Sun Jan 15 10:51:40 2006] [notice] jk2_init() Found child 32235 in scoreboard slot 9
[Sun Jan 15 10:51:40 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:40 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:41 2006] [error] jk2_init() Can't find child 32236 in scoreboard
[Sun Jan 15 10:51:41 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:41 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:42 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
script not found or unable to stat
[Sun Jan 15 10:51:42 2006] [error] jk2_init() Can't find child 32237 in scoreboard
[Sun Jan 15 10:51:42 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:42 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:42 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:42 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
script not found or unable to stat
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:47 2006] [notice] jk2_init() Found child 32238 in scoreboard slot 14
[Sun Jan 15 10:51:47 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:47 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
script not found or unable to stat
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
script not found or unable to stat
[Sun Jan 15 10:51:51 2006] [error] jk2_init() Can't find child 32239 in scoreboard
[Sun Jan 15 10:51:51 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:51 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
script not found or unable to stat
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:56 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:56 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:56 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:57 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:58 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:58 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:59 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:52:00 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:52:03 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:58:51 2006] [error] [client 218.26.189.71] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 11:12:35 2006] [error] [client 61.10.7.15] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 11:42:51 2006] [error] [client 66.173.222.134] File does not exist: /var/www/html/_vti_bin
[Sun Jan 15 11:42:51 2006] [error] [client 66.173.222.134] request failed: URI too long (longer than 8190)
[Sun Jan 15 13:17:22 2006] [error] [client 70.183.102.78] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 13:44:07 2006] [error] [client 68.89.15.188] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 14:34:02 2006] [error] [client 68.203.209.170] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 15:20:57 2006] [error] [client 80.176.231.7] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 16:10:18 2006] [error] [client 24.228.39.92] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 16:28:34 2006] [error] [client 66.53.90.211] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 16:35:46 2006] [error] [client 218.201.254.13] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 17:10:24 2006] [error] [client 65.1.27.237] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 18:17:17 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:17 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:18 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:18 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:18 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:20 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 18:17:21 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 18:17:22 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 18:17:24 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 18:17:25 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 18:17:32 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/blog
[Sun Jan 15 18:17:34 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:35 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/blog
[Sun Jan 15 18:17:39 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/blogs
[Sun Jan 15 18:17:42 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/drupal
[Sun Jan 15 18:17:45 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/phpgroupware
script not found or unable to stat
[Sun Jan 15 18:17:54 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/wordpress
[Sun Jan 15 18:17:55 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 18:17:59 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 18:33:48 2006] [error] [client 12.33.188.222] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:00:24 2006] [error] [client 24.16.60.85] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:09:56 2006] [error] [client 58.210.254.90] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:15:19 2006] [error] [client 204.248.118.90] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:18:25 2006] [error] [client 206.174.28.194] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:53:30 2006] [error] [client 58.22.226.198] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 21:19:42 2006] [error] [client 61.116.186.69] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 00:24:02 2006] [error] [client 218.52.137.194] Directory index forbidden by rule: /var/www/html/
[Mon Jan 16 01:08:33 2006] [error] [client 68.145.115.98] Directory index forbidden by rule: /var/www/html/
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:49 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:49 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:49 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:50 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:50 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:28:19 2006] [error] [client 151.196.29.245] Directory index forbidden by rule: /var/www/html/
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat"""

In [64]:
vectorized_texts = vectorize_texts(context, embedding_model)


In [65]:
summaries = cluster_and_summarize(model_summary, tokenizer_summary,
                                  context, vectorized_texts, num_clusters=3)

/data/anaconda/envs/ptca/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [68]:
question = "What is the error: [error] [client 61.152.90.96]?"

In [69]:
answer = answer_question_based_on_summaries(question, summaries)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Answer: 1. 1. 2. 3. 4. 5. 6. 7. 8. 9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22


## Using Vectorization for Text Summarization
Vectorization is not only useful for ranking answers in a QA task, but also for summarizing large texts like log files. 

Creating a summary or a shorter version of a log file is a practical and efficient approach for several reasons. Firstly, asking questions to a full log can be time-consuming due to the large amount of data that needs to be processed (see code cell below which can take multiple minutes for a longer log file! Execute the code above to have all libraries loaded). This can lead to slower response times and a less efficient system overall.

On the other hand, asking questions to a compact summary is much faster. The system only needs to process a smaller amount of data, which can significantly improve response times.

Moreover, a summarized version of a log file is more user-friendly. Users can quickly get an overview of the information without having to sift through large amounts of data. This makes it easier for users to find the information they need and improves the overall user experience.

Therefore, summarizing log files can lead to a more efficient and user-friendly system.

Here's a simple approach to do this:

- Chunking: As before, split the log file into smaller chunks that the transformer model can handle.

- Vectorization: Convert each chunk into a vector. This will give you a set of vectors that represent the content of your log file.

- Clustering: Use a clustering algorithm like K-means to group the vectors. Each cluster will represent a 'topic' in your log file.

- Summarization: For each cluster, choose the chunk whose vector is closest to the centroid of the cluster as the 'summary' of that topic. You could also consider using a transformer model trained for summarization to further condense these representative chunks.

This approach will give you a summary of the main topics in your log file, which can be very useful for understanding the overall content and identifying any issues or important events. However, keep in mind that this is a very basic approach and there are many more advanced techniques for text summarization out there.


In [4]:
context = """[Sun Jan 15 04:11:23 2006] [notice] Digest: generating secret for digest authentication ...
[Sun Jan 15 04:11:23 2006] [notice] Digest: done
[Sun Jan 15 04:11:23 2006] [notice] LDAP: Built with OpenLDAP LDAP SDK
[Sun Jan 15 04:11:23 2006] [notice] LDAP: SSL support unavailable
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating channel.jni:jni ( channel.jni, jni)
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create channel.jni:jni
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating vm: ( vm, )
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create vm:
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating worker.jni:onStartup ( worker.jni, onStartup)
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create worker.jni:onStartup
[Sun Jan 15 04:11:23 2006] [error] env.createBean2(): Factory error creating worker.jni:onShutdown ( worker.jni, onShutdown)
[Sun Jan 15 04:11:23 2006] [error] config.update(): Can't create worker.jni:onShutdown
[Sun Jan 15 04:11:25 2006] [notice] mod_python: Creating 32 session mutexes based on 150 max processes and 0 max threads.
[Sun Jan 15 04:11:25 2006] [notice] mod_security/1.9dev2 configured
[Sun Jan 15 04:11:25 2006] [notice] Apache/2.0.49 (Fedora) configured -- resuming normal operations
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27313 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27314 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27316 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27317 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27318 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [error] jk2_init() Can't find child 27319 in scoreboard
[Sun Jan 15 04:11:25 2006] [notice] jk2_init() Found child 27315 in scoreboard slot 13
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 04:11:25 2006] [notice] jk2_init() Found child 27320 in scoreboard slot 1
[Sun Jan 15 04:11:25 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 04:11:25 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 04:17:29 2006] [error] [client 222.78.70.94] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 04:25:54 2006] [error] [client 68.193.179.224] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 05:29:33 2006] [error] [client 203.196.138.5] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 06:23:20 2006] [error] [client 66.214.157.114] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 06:53:16 2006] [error] [client 218.204.92.66] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 07:56:06 2006] [error] [client 64.60.3.98] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 08:48:10 2006] [error] [client 69.220.16.126] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 10:51:33 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:33 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] jk2_init() Can't find child 32229 in scoreboard
[Sun Jan 15 10:51:34 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:34 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:34 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:35 2006] [notice] jk2_init() Found child 32230 in scoreboard slot 2
[Sun Jan 15 10:51:35 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:35 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:35 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:36 2006] [error] jk2_init() Can't find child 32231 in scoreboard
[Sun Jan 15 10:51:36 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:36 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:36 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:37 2006] [error] jk2_init() Can't find child 32232 in scoreboard
[Sun Jan 15 10:51:37 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:37 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:37 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
[Sun Jan 15 10:51:38 2006] [notice] jk2_init() Found child 32233 in scoreboard slot 6
[Sun Jan 15 10:51:38 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:38 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:38 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:39 2006] [error] jk2_init() Can't find child 32234 in scoreboard
[Sun Jan 15 10:51:39 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:39 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
script not found or unable to stat
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
script not found or unable to stat
[Sun Jan 15 10:51:39 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
[Sun Jan 15 10:51:40 2006] [notice] jk2_init() Found child 32235 in scoreboard slot 9
[Sun Jan 15 10:51:40 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:40 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:40 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:41 2006] [error] jk2_init() Can't find child 32236 in scoreboard
[Sun Jan 15 10:51:41 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:41 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:41 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:42 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
script not found or unable to stat
[Sun Jan 15 10:51:42 2006] [error] jk2_init() Can't find child 32237 in scoreboard
[Sun Jan 15 10:51:42 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:42 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:42 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:42 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
script not found or unable to stat
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:43 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:44 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
script not found or unable to stat
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:45 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:46 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:47 2006] [notice] jk2_init() Found child 32238 in scoreboard slot 14
[Sun Jan 15 10:51:47 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:47 2006] [error] mod_jk child workerEnv in error state 6
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:47 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blog
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:48 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
script not found or unable to stat
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:49 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:50 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
script not found or unable to stat
[Sun Jan 15 10:51:51 2006] [error] jk2_init() Can't find child 32239 in scoreboard
[Sun Jan 15 10:51:51 2006] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Jan 15 10:51:51 2006] [error] mod_jk child init 1 -2
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:51 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:52 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/blogs
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:53 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/drupal
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
script not found or unable to stat
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:54 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:55 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 10:51:56 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/wordpress
[Sun Jan 15 10:51:56 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:56 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:57 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 10:51:58 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:51:58 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:51:59 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 10:52:00 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:52:03 2006] [error] [client 61.152.90.96] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 10:58:51 2006] [error] [client 218.26.189.71] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 11:12:35 2006] [error] [client 61.10.7.15] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 11:42:51 2006] [error] [client 66.173.222.134] File does not exist: /var/www/html/_vti_bin
[Sun Jan 15 11:42:51 2006] [error] [client 66.173.222.134] request failed: URI too long (longer than 8190)
[Sun Jan 15 13:17:22 2006] [error] [client 70.183.102.78] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 13:44:07 2006] [error] [client 68.89.15.188] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 14:34:02 2006] [error] [client 68.203.209.170] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 15:20:57 2006] [error] [client 80.176.231.7] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 16:10:18 2006] [error] [client 24.228.39.92] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 16:28:34 2006] [error] [client 66.53.90.211] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 16:35:46 2006] [error] [client 218.201.254.13] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 17:10:24 2006] [error] [client 65.1.27.237] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 18:17:17 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:17 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:18 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:18 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:18 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:20 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 18:17:21 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 18:17:22 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 18:17:24 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 18:17:25 2006] [error] [client 220.134.161.152] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 18:17:32 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/blog
[Sun Jan 15 18:17:34 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 18:17:35 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/blog
[Sun Jan 15 18:17:39 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/blogs
[Sun Jan 15 18:17:42 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/drupal
[Sun Jan 15 18:17:45 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/phpgroupware
script not found or unable to stat
[Sun Jan 15 18:17:54 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/wordpress
[Sun Jan 15 18:17:55 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 18:17:59 2006] [error] [client 220.134.161.152] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 18:33:48 2006] [error] [client 12.33.188.222] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:00:24 2006] [error] [client 24.16.60.85] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:09:56 2006] [error] [client 58.210.254.90] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:15:19 2006] [error] [client 204.248.118.90] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:18:25 2006] [error] [client 206.174.28.194] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 19:53:30 2006] [error] [client 58.22.226.198] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 21:19:42 2006] [error] [client 61.116.186.69] Directory index forbidden by rule: /var/www/html/
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:13 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/awstats/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:14 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:15 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
[Sun Jan 15 23:31:16 2006] [error] [client 149.132.103.59] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:19 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:20 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blog
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:21 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:22 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/blogs
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:23 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/drupal
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:24 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/phpgroupware
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
[Sun Jan 15 23:31:25 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/wordpress
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:27 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:28 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlrpc
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Sun Jan 15 23:31:29 2006] [error] [client 149.132.103.59] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 00:24:02 2006] [error] [client 218.52.137.194] Directory index forbidden by rule: /var/www/html/
[Mon Jan 16 01:08:33 2006] [error] [client 68.145.115.98] Directory index forbidden by rule: /var/www/html/
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:34 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:35 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:36 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:37 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 01:08:38 2006] [error] [client 66.34.225.128] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:39 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:40 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:41 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blog
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:42 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/blogs
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:43 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/drupal
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:44 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
[Mon Jan 16 01:08:45 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/phpgroupware
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:46 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/wordpress
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:47 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:48 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:49 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlrpc
[Mon Jan 16 01:08:49 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:49 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:50 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:08:50 2006] [error] [client 66.34.225.128] File does not exist: /var/www/html/xmlsrv
[Mon Jan 16 01:28:19 2006] [error] [client 151.196.29.245] Directory index forbidden by rule: /var/www/html/
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:09 2006] [error] [client 82.238.44.122] File does not exist: /var/www/html/awstats/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:10 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats.pl
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
[Mon Jan 16 02:09:11 2006] [error] [client 82.238.44.122] script not found or unable to stat: /var/www/cgi-bin/awstats
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat
script not found or unable to stat"""

In [8]:
#question = "What error message is displayed?"
question = "Which file does not exist?"

# Find the best answer to the question given the context
best_answer = find_best_answer(model, tokenizer, context, question, model_vec, num_answers=3, max_length=tokenizer.model_max_length)
print(f"The best answer is: {best_answer}")

Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (588) must match the size of tensor b (512) at non-singleton dimension 1